Text Summarizer using T5-small model 📝
===

## 1- Overview
In this project, we explored transformer-based models for text summarization. We specifically utilized the T5-Small model to summarize news articles. The T5-Small model was chosen due to its lightweight architecture, which makes it suitable for shorter texts and environments with limited computational resources. To optimize performance, we sampled the dataset to align with the available computational power.


## 2- Dataset Used
We used a news dataset containing two main columns:
- Text – the full body of the news article.
- Headline – the target summary representing the expected output.

## 3- Data Cleaning
The dataset underwent several cleaning steps to ensure quality and consistency:
- Removed duplicate records.
- Eliminated URLs, special characters, and extra whitespaces from the text.
- Split the dataset into training, validation, and testing sets to enhance performance evaluation.

## 4- Data Preprocessing
We tokenized the text data into numerical representations that the T5 model can process effectively. This step ensures that both input texts and target summaries are appropriately encoded before training.

## 5- Modeling
We implemented and fine-tuned the T5-Small model using the following approach:
- Defined the model architecture and training parameters.
- Chose ROUGE as the evaluation metric to assess summarization quality.
- Trained the model for 3 epochs, monitoring training and validation performance.

# Required Installation

In [1]:
# Set environment variable for memory fragmentation
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

In [2]:
pip install transformers torch rouge-score pandas scikit-learn matplotlib datasets evaluate accelerate sentencepiece 

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 33.8 MB/s eta 0:00:00
 

In [3]:
!pip install -U transformers datasets sentencepiece accelerate evaluate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 92.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.8/375.8 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 84.6 MB/s eta 0:00:00


# Import Libraries

In [4]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [5]:
# import required libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import(
    T5Tokenizer, 
    T5ForConditionalGeneration,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq
)
import evaluate
import torch
import re
import random


2025-11-07 12:38:15.610090: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762519095.802903      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762519095.862280      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Data Loading

In [6]:
# dataset loading
data = pd.read_csv('/kaggle/input/news-summary/news_summary_more.csv')
data.head()


,headlines,text
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...


In [7]:
# data shape
data.shape

(98401, 2)

In [8]:
# display the statistical summary
data.describe(include='object')

,headlines,text
count,98401,98401
unique,98280,98360
top,Warne rightly predicted India-Eng 2011 WC matc...,Former Pakistani cricketer Hanif Mohammad batt...
freq,3,2


In [9]:
# checking for missing values
data.isnull().sum()

headlines    0
text         0
dtype: int64

In [10]:
# checking for duplications
data.duplicated().sum()

22

In [11]:
# drop duplicates
data.drop_duplicates(inplace = True)

In [12]:
# sampling the data
df = data.sample(n = 7000, random_state = 42)
df = df.reset_index(drop = True)

In [13]:
# displaying the max-text-length, and max-headline-length
headline_length = df['headlines'].str.len()
text_length = df['text'].str.len()
print('Max Headline Length: ', max(headline_length))
print('Max Text Length: ', max(text_length))

Max Headline Length:  79
Max Text Length:  431


# Data Cleaning

In [14]:
# data cleaning
def clean_text(text):
    # Remove special characters
    text = re.sub(r'[^a-zA-Z\s]', "", text)
    # remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove extra whitespaces
    text = ' '.join(text.split())
    return text

df['text'] = df['text'].map(clean_text)
df['headlines'] = df['headlines'].map(clean_text)

# remove empty or too short samples
df = df[(df['text'].str.len() > 50) & (df['headlines'].str.len() > 10)]
print('After cleaning: ', df.shape)

After cleaning:  (7000, 2)


In [15]:
df.head()

,headlines,text
0,Exdentists new mobile app gets mn after failed...,Former South Korean dentist Seunggun Lees mobi...
1,LG tests robots to guide passengers in airport...,LG is testing two robot prototypes to provide ...
2,Opposition is trying to kill me Delhi CM on ch...,Reacting to the chilli powder attack on him on...
3,Policeman touches UP Ministers feet for scrapi...,A policeman in Uttar Pradeshs Kanpur touched t...
4,Id love to make Mahabharat film but dont have ...,Actor Shah Rukh Khan has said that he would lo...


# Data Splitting

In [16]:
# data splitting into train, validation, and testing sets
train_df, temp_df = train_test_split(df, test_size = 0.2, random_state = 42)
val_df, test_df = train_test_split(temp_df, test_size = 0.5, random_state = 42)

print(f'Train: {len(train_df)} | Val: {len(val_df)} | Test: {len(test_df)}')

Train: 5600 | Val: 700 | Test: 700


In [17]:
# converting into Hugging Face data format
train_ds = Dataset.from_pandas(train_df)
val_ds = Dataset.from_pandas(val_df)
test_ds = Dataset.from_pandas(test_df)

# Tokenization & Preprocessing

In [18]:
# define the model and tokenizer
model_name = 'T5-small'
tokenizer = T5Tokenizer.from_pretrained(model_name)

max_input_length = 512
max_target_length = 64

# data preprocessing
def preprocess_function(examples):
    inputs = ['summarize: '+ doc for doc in examples['text']]
    model_inputs = tokenizer(inputs, max_length = max_input_length,
                            truncation = True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["headlines"], max_length=max_target_length,
                           truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_ds.map(preprocess_function, batched=True, remove_columns=["text", "headlines"])
tokenized_val = val_ds.map(preprocess_function, batched=True, remove_columns=["text", "headlines"])
tokenized_test = test_ds.map(preprocess_function, batched=True, remove_columns=["text", "headlines"])

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Map:   0%|          | 0/5600 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/700 [00:00<?, ? examples/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

# Load Model

In [19]:
import torch
import gc

# Clear GPU cache
torch.cuda.empty_cache()
gc.collect()

# Check GPU memory
print(f"GPU memory allocated: {torch.cuda.memory_allocated(0)/1024**3:.2f} GB")
print(f"GPU memory reserved: {torch.cuda.memory_reserved(0)/1024**3:.2f} GB")

GPU memory allocated: 0.00 GB
GPU memory reserved: 0.00 GB


In [20]:
# load the model
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.gradient_checkpointing_enable() 


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

# Evaluation Metrics (ROUGE)

In [21]:
rouge = evaluate .load('rouge')

def compute_metrics(eval_pred):
    preds, labels = eval_pred
    
    # Convert logits to token IDs 
    if isinstance(preds, tuple):
        preds = preds[0]
    
    # If preds are 3D (batch, sequence, vocab), convert to token IDs
    if len(preds.shape) == 3:
        preds = np.argmax(preds, axis=-1)
    
    # Decode predictions
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 in labels (used for padding)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Compute ROUGE scores
    result = rouge.compute(
        predictions=decoded_preds,
        references=decoded_labels,
        use_stemmer=True
    )
    
    # Return scores 
    return {
        "rouge1": result["rouge1"],
        "rouge2": result["rouge2"],
        "rougeL": result["rougeL"]
    }

# Defining Training Arguments

In [22]:
training_args = TrainingArguments(
    output_dir="./results",
    
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=3e-5,
    weight_decay=0.01,
    
    eval_strategy="epoch",  
    save_strategy="epoch",  
    
    logging_dir="./logs",
    logging_strategy="steps",
    logging_steps=100,
    
    fp16=True,
    gradient_accumulation_steps=4,  
    gradient_checkpointing=True, 
    optim="adamw_torch",
    
    push_to_hub=False,
    report_to="none",
    
    dataloader_pin_memory=False  
)

# Initialize Trainer

In [23]:
# DataCollator handles dynamic padding
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/tmp/ipykernel_19/3767616073.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [24]:
# Checking the disk space before training
import shutil
disk_usage = shutil.disk_usage("./")
free_gb = disk_usage.free / (1024**3)
print(f"Free disk space: {free_gb:.2f} GB")

if free_gb < 5:
    print("WARNING: Less than 5GB free space. Consider freeing up space or saving to external storage.")


Free disk space: 19.50 GB


# Train Model

In [25]:
train_result = trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel
1,2.627700,2.223325,0.443935,0.222748,0.431589
2,2.462900,2.126710,0.452636,0.237667,0.442232
3,2.405800,2.103310,0.455484,0.239223,0.444745


In [26]:
# After training
torch.cuda.empty_cache()
gc.collect()

619

# Evaluation

In [27]:
# Print training results
print("\nTraining completed!")
print(f"Training time: {train_result.metrics['train_runtime']:.2f} seconds")
print(f"Training samples/second: {train_result.metrics['train_samples_per_second']:.2f}")

# Evaluate on Validation Set 
# ============================================================================
print("\n" + "=" * 80)
print("Evaluating on validation set... ")
print("=" * 80)

val_results = trainer.evaluate(tokenized_val)

print("\nValidation Set Results:")
for key, value in val_results.items():
    print(f"{key}: {value:.4f}")

# Evaluate on Test Set
# ============================================================================
print("\n" + "=" * 80)
print("Evaluating on Test Set... ")
print("=" * 80)

test_results = trainer.evaluate(tokenized_test)

print("\nTest Set Results:")
for key, value in test_results.items():
    print(f"{key}: {value:.4f}")

# Save the Model
# ============================================================================
print("\n" + "=" * 80)
print("STEP 14: Saving Model")
print("=" * 80)

# Save model and tokenizer
model.save_pretrained("/kaggle/working/t5-summarization-model")
tokenizer.save_pretrained("/kaggle/working/t5-summarization-model")

print("Model saved to '/kaggle/working/t5-summarization-model'")



Training completed!
Training time: 405.66 seconds
Training samples/second: 41.41

Evaluating on validation set... 



Validation Set Results:
eval_loss: 2.1033
eval_rouge1: 0.4555
eval_rouge2: 0.2392
eval_rougeL: 0.4447
eval_runtime: 11.6371
eval_samples_per_second: 60.1520
eval_steps_per_second: 15.0380
epoch: 3.0000

Evaluating on Test Set... 

Test Set Results:
eval_loss: 2.1732
eval_rouge1: 0.4421
eval_rouge2: 0.2299
eval_rougeL: 0.4336
eval_runtime: 11.5302
eval_samples_per_second: 60.7100
eval_steps_per_second: 15.1780
epoch: 3.0000

STEP 14: Saving Model
Model saved to '/kaggle/working/t5-summarization-model'


# Generating Summaries

In [28]:
def generate_summary(text, max_length=64):
    input_text = "summarize: " + text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)
    
    # Move to device
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_beams=4,
        length_penalty=2.0,
        early_stopping=True
    )
    
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [29]:
# Test on random samples
for i in random.sample(range(len(test_df)), 5):
    sample = test_df.iloc[i]
    generated = generate_summary(sample['text'])
    
    print(f"\n{'='*80}")
    print(f"📄 Original Text ({len(sample['text'])} chars):")
    print(f"{sample['text'][:300]}...")
    print(f"\n🎯 Actual Headline: {sample['headlines']}")
    print(f"🤖 Generated Summary: {generated}")
    print(f"{'='*80}")


📄 Original Text (363 chars):
USbased scientists are developing a new system that can predict if a car driver is about to have a heart attack Researchers in collaboration with Japanese automaker Toyota are working on technology that can monitor and analyse the physiology of the driver The medical event interferes with the driver...

🎯 Actual Headline: Cars may predict if driver is having a heart attack
🤖 Generated Summary: US scientists developing system to predict if car driver will have heart attack

📄 Original Text (346 chars):
Actors Salman Khan Katrina Kaif Alia Bhatt and Varun Dhawan were among the celebrities who attended the IIFA Rocks event in New York Katrina wore a ruffled saree and golden tassel blouse by Abu Jani Sandeep Khosla Alia chose a cream coloured lehenga by Manish Malhotra Meanwhile Taapsee Pannu paired ...

🎯 Actual Headline: Salman Katrina Alia attend IIFA Rocks
🤖 Generated Summary: Salman Khan Katrina Kaif Alia Bhatt and Varun Dhawan among celebrities who atten